# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [1]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np
#load data
strwb = pd.read_csv("strawberry-prices.tsv", sep='\t')
#conver months col. to date-time
strwb['month'] = pd.to_datetime(strwb['month'])
#separate into train
train = strwb[strwb['month'] <= '2023-12-31'] #only want 2020-2023
#get calendar month (1-12)
train['month_num'] = train['month'].dt.month
#compute avg. price per month
monthly_avg = (train.groupby("month_num")['price'].mean())
#create 2024 months
months_24 = pd.date_range(start='2024-01-01', end='2024-12-31', freq='MS')
#predict prices using monthly avgs
pred_prices = [monthly_avg[m.month] for m in months_24]
#save backtest
bktest_24 = pd.DataFrame({"month": months_24, "price": pred_prices})
bktest_24

,month,price
0,2024-01-01,4.362750
1,2024-02-01,4.091250
2,2024-03-01,3.688750
3,2024-04-01,3.787200
4,2024-05-01,3.526250
5,2024-06-01,3.275000
6,2024-07-01,3.194000
7,2024-08-01,3.489750
8,2024-09-01,3.581750
9,2024-10-01,3.926000


In [2]:
#save to tsv file
bktest_24.to_csv("strawberry-backtest.tsv", sep="\t", index=False)

Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [3]:
# YOUR CHANGES HERE
pred_24 = bktest_24
truth_24 = strwb[strwb['month'].dt.year == 2024] #only want vals from 2024
#merge pred and truth
mg_24 = pred_24.merge(truth_24, on='month', suffixes=('_pred', '_actual'))
#compute residuals
mg_24['residual'] = mg_24['price_actual'] - mg_24['price_pred']
#compute mean and std
mean_24 = mg_24['residual'].mean()
std_24 = mg_24['residual'].std()
#add to df
stats_24 = pd.DataFrame({"mean": [mean_24], "std": [std_24]})
#convert to tsv
stats_24.to_csv("backtest-accuracy.tsv", sep='\t', index=False)

Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [4]:
# YOUR CHANGES HERE
#separate into train
tr_25 = strwb[strwb['month'] <= '2024-12-31'] #only want 2020-2024
#get calendar month (1-12)
tr_25['month_num'] = tr_25['month'].dt.month
#compute avg. price per month
mo_avg_25 = (tr_25.groupby("month_num")['price'].mean())
#create 2024 months
mos_25 = pd.date_range(start='2025-01-01', end='2025-12-31', freq='MS')
#predict prices using monthly avgs
pred_prices_25 = [mo_avg_25[m.month] for m in mos_25]
#save backtest
bktest_25 = pd.DataFrame({"month": mos_25, "price": pred_prices_25})
bktest_25

,month,price
0,2025-01-01,4.50120
1,2025-02-01,4.12580
2,2025-03-01,3.69940
3,2025-04-01,3.74496
4,2025-05-01,3.46840
5,2025-06-01,3.21540
6,2025-07-01,3.17840
7,2025-08-01,3.46120
8,2025-09-01,3.61380
9,2025-10-01,3.88440


In [5]:
#save to tsv file
bktest_25.to_csv("strawberry-forecast.tsv", sep="\t", index=False)

Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [6]:
# YOUR CHANGES HERE
capital = 1000000 #intial amount to invest
frz_cost = 0.20 #freeze cost per pint
stg_cost = 0.10 #storage cost per pint
discount = 0.9 #10% price discount leaves us w/90%
results = [] #store results here
for i, buy_row in bktest_25.iterrows():
    for j, sell_row in bktest_25.iterrows():
        if j <= i:
            continue
        buy_month = buy_row['month']
        sell_month = sell_row['month']
        buy_price = buy_row['price']
        sell_price = sell_row['price']
        #months stored
        mos_stored = (sell_month.year - buy_month.year) * 12 + (sell_month.month - buy_month.month)
        #calculate per pint
        cost_p_pint = buy_price + frz_cost + stg_cost * mos_stored
        rev_p_pint = sell_price * discount #revenue per pint
        profit_p_pint = rev_p_pint - cost_p_pint
        #max pints able to purchase
        pints = int(capital//cost_p_pint)
        expected_profit = pints*profit_p_pint
        results.append({
            "buy_month": buy_month.strftime("%Y-%m"),
            "sell_month": sell_month.strftime("%Y-%m"),
            "pints_purchased": pints,
            "expected_profit": expected_profit
        })

#create df w/results
df = pd.DataFrame(results)
#sort by best profit
df = df.sort_values("expected_profit", ascending=False) #for easier readability 
df

,buy_month,sell_month,pints_purchased,expected_profit
55,2025-07,2025-12,257838,114385.703292
50,2025-06,2025-12,249041,76364.437994
59,2025-08,2025-12,246232,64225.677488
62,2025-09,2025-12,243084,50618.353656
54,2025-07,2025-11,264662,48634.289120
...,...,...,...,...
15,2025-02,2025-07,207219,-407235.067560
7,2025-01,2025-09,181778,-408778.730840
6,2025-01,2025-08,185144,-423261.401280
4,2025-01,2025-06,192263,-443616.110420


In [7]:
#convert to tsv
df.to_csv("timings.tsv", sep='\t', index=False)

Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [8]:
# YOUR CHANGES HERE
#find best profit
best_row = df.loc[df['expected_profit'].idxmax()]
best_profit = best_row['expected_profit']
pints_purch = best_row['pints_purchased']
#use residual stats from p3: std_24, mean_24
#one std impact on profit
one_std_prof = pints_purch * std_24
#save results in df
check = pd.DataFrame({"best_profit": [best_profit], "one_std_profit": [one_std_prof]})
check

,best_profit,one_std_profit
0,114385.703292,70793.566481


In [9]:
#save to tsv
check.to_csv("check.tsv", sep='\t', index=False)

Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [10]:
file_content = "none"
with open('acknowledgements.txt', 'w') as f:
    f.write(file_content)

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.